In [4]:
%matplotlib inline
import d2lzh as d2l
from mxnet import autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,nn

In [6]:
'''生成数据集'''
#y = 1.2x − 3.4x2+ 5.6x + 5 + ϵ,
#其中噪声ϵ服从均值为0，标准查为0.1的正态分布，训练数据集和测试数据集的样本数都设为100
n_train,n_test,true_w,true_b = 100,100,[1.2,-3.4,5.6],5
#单列数据
features = nd.random.normal(shape=(n_train + n_test,1))
#三列数据
poly_features = nd.concat(features,nd.power(features,2),nd.power(features,3))
#得到标签
#这是NDArray的取某列，和python的区间选择区分开
#这里的某列最终表现为横排数组，最终得到一个横排的结果，并不是矩阵乘法
labels = ( true_w[0] * poly_features[:,0] + true_w[1] * poly_features[:,1] + true_w[2] * poly_features[:,2] + true_b)
#加上噪音,噪音值服从均值为0、标准差为0.1的正态分布
labels = nd.random.normal(scale=0.1,shape=labels.shape)

#查看数据集合的前两个样本
#python的取前几个值
features[:2],poly_features[:2],labels[:2]
'''
取第一列的三个值横着排，乘以第一个权重，
取第二列的三个值横着排，乘以第二个权重
取第三列的三个值横着排，乘以第三个权重
最后三个数组相加，得到一个横排的三个结果

'''

(
 [[-0.6768301 ]
  [-0.61593443]]
 <NDArray 2x1 @cpu(0)>,
 
 [[-0.6768301   0.458099   -0.3100552 ]
  [-0.61593443  0.37937522 -0.23367026]]
 <NDArray 2x3 @cpu(0)>,
 
 [-0.00340771 -0.08689129]
 <NDArray 2 @cpu(0)>)